<h1><center>Laboratorio 8: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza y Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano y Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Daniel Minaya
- Nombre de alumno 2:


### **Link de repositorio de GitHub:** `https://github.com/DanielMinaya1/MDS7202`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer material del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder

# Modelos de clasificación
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# Modelos de regresión
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.svm import SVR

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import r2_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.csv`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [3]:
df_players = pd.read_csv('stats_players.csv')
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [4]:
# Etiquetamos True si fue seleccionado y False si no
df_players['Labels']  = ~df_players['National_Position'].isna()
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Labels
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,95,85,92,93,90,81,76,85,88,True
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,68,71,85,95,88,89,90,74,85,True
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,61,62,78,89,77,79,84,81,83,True
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,69,77,87,94,86,86,84,85,88,True
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,78,25,25,13,16,14,11,47,11,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,56,14,16,13,13,11,13,15,12,False
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,53,12,17,12,11,12,13,16,12,False
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,61,41,44,28,42,35,36,42,37,False
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,48,15,23,14,12,13,12,24,12,False


### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>
<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [X] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [X] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [X] Crear 3 ``pipelines`` con diferentes clasificadores.
- [X] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [X] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

In [5]:
# Agrupamos las columnas con su tipo de procesamiento
standard_columns = ['Height', 'Weight']
minmax_columns = ['Age', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle', 'Aggression', 
                  'Reactions', 'Interceptions', 'Vision', 'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 
                  'Acceleration', 'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping', 'Heading', 
                  'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy', 'Penalties', 'Volleys']
onehot_columns = ['Club_Position', 'Preffered_Foot']
ordinal_columns = ['Work_Rate', 'Weak_foot', 'Skill_Moves']

# Definimos el orden de las variables ordinales
ordinal_categories = [
    ['Low / Low', 'Low / Medium', 'Low / High', 
     'Medium / Low', 'Medium / Medium', 'Medium / High',
     'High / Low', 'High / Medium', 'High / High'], 
    [1, 2, 3, 4, 5], 
    [1, 2, 3, 4, 5]]

# Definimos el ColumnTransformer
preprocessing = ColumnTransformer(
    transformers = [
        ('standard', StandardScaler(), standard_columns),
        ('minmax', MinMaxScaler(), minmax_columns),
        ('onehot', OneHotEncoder(), onehot_columns),
        ('ordinal', OrdinalEncoder(categories=ordinal_categories), ordinal_columns),
    ],
    remainder = 'drop'
)

# Preprocesamos
preprocessed_data = preprocessing.fit_transform(df_players)

# Obtenemos el nombre de las columnas onehot
onehot_columns = preprocessing.named_transformers_['onehot'].get_feature_names_out(onehot_columns).tolist()

# Pasamos a dataframe
preprocessed_df = pd.DataFrame(preprocessed_data, 
                               columns = standard_columns + minmax_columns + onehot_columns + ordinal_columns)
preprocessed_df

,Height,Weight,Age,Ball_Control,Dribbling,Marking,Sliding_Tackle,Standing_Tackle,Aggression,Reactions,...,Club_Position_RWB,Club_Position_Res,Club_Position_ST,Club_Position_Sub,Club_Position_nan,Preffered_Foot_Left,Preffered_Foot_Right,Work_Rate,Weak_foot,Skill_Moves
0,0.583453,0.688144,0.500000,0.977778,0.946237,0.213483,0.200000,0.314607,0.648936,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,3.0,4.0
1,-1.663748,-0.471654,0.400000,1.000000,1.000000,0.112360,0.233333,0.280899,0.489362,0.985075,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,3.0,3.0
2,-1.064495,-1.051553,0.266667,1.000000,0.989247,0.202247,0.311111,0.235955,0.574468,0.880597,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7.0,4.0,4.0
3,0.134013,1.413018,0.433333,0.955556,0.881720,0.303371,0.366667,0.471910,0.808511,0.955224,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,7.0,3.0,3.0
4,1.781960,2.427841,0.466667,0.477778,0.279570,0.078652,0.066667,0.078652,0.287234,0.835821,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,0.283826,0.978093,0.066667,0.077778,0.075269,0.101124,0.077778,0.112360,0.244681,0.283582,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,4.0,0.0,0.0
17584,0.583453,0.688144,0.066667,0.088889,0.075269,0.089888,0.077778,0.089888,0.138298,0.208955,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,4.0,1.0,0.0
17585,-1.214308,-2.066377,0.033333,0.433333,0.376344,0.415730,0.266667,0.359551,0.446809,0.194030,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,1.0,1.0
17586,-0.165614,0.688144,0.133333,0.133333,0.096774,0.089888,0.066667,0.123596,0.223404,0.253731,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,4.0,2.0,0.0


>Justifiquemos la elección de las transformaciones utilizadas.
>
>**Para las variables numéricas:** Viendo las distribuciones de <code>Height</code>, <code>Weight</code> podemos notar que son similares a una distribución normal, por lo que utilizaremos <code>StandardScaler</code>. También vemos que la variable <code>Age</code> se asemeja a una normal, sin embargo, en este caso utilizaremos <code>MinMaxScaler</code>, pues el rango de edades de los jugadores es una información que puede ser útil para los modelos y usando esta transformación lo mantenemos. Para el resto de variables utilizaremos <code>MinMaxScaler</code> debido a que son atributos dentro de un rango entre 0 y 100, independiente de que alguna de estas variables pueda tener una distribución similar a una normal.
>
>**Para las variables categóricas:** Eliminamos la variable <code>Name</code> y <code>Nationality</code> porque tienen demasiados valores únicos y no aportan información relevante a la predicción que queremos realizar, además la variable <code>Name</code> está distribuida de manera uniforme. También se elimina la variable <code>National_Position</code> pues es la columna de la cual obtuvimos las etiquetas. Con respecto al resto de variables: <code>Club_Position</code> y <code>Preffered_Foot</code> se procesan con OneHotEncoder porque son variables categóricas que no presentan un orden particular. <code>Work_Rate</code>, <code>Weak_foot</code> y <code>Skill_Moves</code> se procesan con OrdinalEncoder pues son variables categóricas con un orden asociado. 

In [6]:
# Definimos las pipelines con los clasificadores (agregamos DecisionTree porque hay dos SVC)
pipeline_1 = Pipeline([
    ('preprocessing', preprocessing), 
    ('classifier', LinearSVC(dual='auto', random_state=42))
])

pipeline_2 = Pipeline([
    ('preprocessing', preprocessing), 
    ('classifier', KNeighborsClassifier())
])

pipeline_3 = Pipeline([
    ('preprocessing', preprocessing), 
    ('classifier', SVC(random_state=42))
])

pipeline_4 = Pipeline([
    ('preprocessing', preprocessing), 
    ('classifier', DecisionTreeClassifier(random_state=42))
])

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [X] Separar el conjunto de datos en entrenamiento y de prueba.
- [X] Entrenar los `pipelines`.
- [X] Utilizar las métricas para evaluar los modelos generados.


In [7]:
# Separamos el conjunto de datos (usamos stratify para mantener la distribución de labels)
features = df_players.drop(columns=["Labels"])
labels = df_players.loc[:, "Labels"]

X_train, X_test, y_train, y_test = train_test_split(
    features, labels, 
    test_size = 0.3, 
    stratify = labels,
    shuffle = True,
    random_state = 42,
)

In [8]:
# Entrenamos las pipelines
pipeline_1.fit(X_train, y_train)
pipeline_2.fit(X_train, y_train)
pipeline_3.fit(X_train, y_train)
pipeline_4.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('standard', StandardScaler(),
                                                  ['Height', 'Weight']),
                                                 ('minmax', MinMaxScaler(),
                                                  ['Age', 'Ball_Control',
                                                   'Dribbling', 'Marking',
                                                   'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Aggression', 'Reactions',
                                                   'Interceptions', 'Vision',
                                                   'Composure', 'Crossing',
                                                   'Short_Pass', 'Long_Pass',
                                                   'Acceleration', 'Speed',
                                                   'Stamina',...
                                                  ['Club_Position',
                                                   'Preffered_Foot']),
                                                 ('ordinal',
                                                  OrdinalEncoder(categories=[['Low '
                                                                              '/ '
                                                                              'Low',
                                                                              'Low '
                                                                              '/ '
                                                                              'Medium',
                                                                              'Low '
                                                                              '/ '
                                                                              'High',
                                                                              'Medium '
                                                                              '/ '
                                                                              'Low',
                                                                              'Medium '
                                                                              '/ '
                                                                              'Medium',
                                                                              'Medium '
                                                                              '/ '
                                                                              'High',
                                                                              'High '
                                                                              '/ '
                                                                              'Low',
                                                                              'High '
                                                                              '/ '
                                                                              'Medium',
                                                                              'High '
                                                                              '/ '
                                                                              'High'],
                                                                             [1,
                                                                              2,
                                                                              3,
                                                                              4,
                                                                              5],
                                                                             [1,
         

In [9]:
# Predecimos 
predictions_1 = pipeline_1.predict(X_test)
predictions_2 = pipeline_2.predict(X_test)
predictions_3 = pipeline_3.predict(X_test)
predictions_4 = pipeline_4.predict(X_test)

In [10]:
for i in range(1, 5):
    predictions = globals()[f'predictions_{i}']
    print(f"Accuracy para pipeline_{i}: {accuracy_score(y_test, predictions)}")
    print(f"F1-score para pipeline_{i}: {f1_score(y_test, predictions)}")
    print(f"Cohen kappa para pipeline_{i}: {cohen_kappa_score(y_test, predictions)}\n")

Accuracy para pipeline_1: 0.9387909797233277
F1-score para pipeline_1: 0.0
Cohen kappa para pipeline_1: 0.0

Accuracy para pipeline_2: 0.9368959636156907
F1-score para pipeline_2: 0.061971830985915494
Cohen kappa para pipeline_2: 0.051505319639891245

Accuracy para pipeline_3: 0.9387909797233277
F1-score para pipeline_3: 0.0
Cohen kappa para pipeline_3: 0.0

Accuracy para pipeline_4: 0.9056281978396816
F1-score para pipeline_4: 0.26112759643916916
Cohen kappa para pipeline_4: 0.21081565139677683



In [11]:
print(classification_report(y_test, predictions_1))

              precision    recall  f1-score   support

       False       0.94      1.00      0.97      4954
        True       0.00      0.00      0.00       323

    accuracy                           0.94      5277
   macro avg       0.47      0.50      0.48      5277
weighted avg       0.88      0.94      0.91      5277



C:\Users\Daniel Minaya Vargas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Daniel Minaya Vargas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Daniel Minaya Vargas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
print(classification_report(y_test, predictions_2))

              precision    recall  f1-score   support

       False       0.94      1.00      0.97      4954
        True       0.34      0.03      0.06       323

    accuracy                           0.94      5277
   macro avg       0.64      0.51      0.51      5277
weighted avg       0.90      0.94      0.91      5277



In [13]:
print(classification_report(y_test, predictions_3))

              precision    recall  f1-score   support

       False       0.94      1.00      0.97      4954
        True       0.00      0.00      0.00       323

    accuracy                           0.94      5277
   macro avg       0.47      0.50      0.48      5277
weighted avg       0.88      0.94      0.91      5277



C:\Users\Daniel Minaya Vargas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Daniel Minaya Vargas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Daniel Minaya Vargas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
print(classification_report(y_test, predictions_4))

              precision    recall  f1-score   support

       False       0.95      0.95      0.95      4954
        True       0.25      0.27      0.26       323

    accuracy                           0.91      5277
   macro avg       0.60      0.61      0.61      5277
weighted avg       0.91      0.91      0.91      5277



>Podemos ver que los cuatros modelos entrenados presentan buenas métricas para la clase False, es decir, predicen bien cuando un jugador no será convocado por su selección, sin embargo, esto no ocurre para la clase True, donde las métricas no llegan al 0.3 ni en precision, recall o f1-score. Además, podemos notar que estas métricas para macro average son mucho peores, siendo el Decision Tree el modelo con mejores métricas. Esta diferencia grande entre el desempeño de los modelos para cada clase se debe a la cantidad de elementos que hay por clase, pues vemos que hay 4954 datos para False y solo 323 para True, es decir, el conjunto de datos está desbalanceado, por lo que una forma de mejorar el desempeño del modelo sería utilizar oversampling, es decir, repetir muestras de True para aumentar de manera artificial la cantidad de datos para esta clase, sin embargo al realizar esto estamos interviniendo la distribución real de las clases, la cual tiene sentido pues en realidad la cantidad de jugadores es demasiado alta para que la gran mayoría sea elegido para su selección. Por otro lado, podríamos utilizar subsampling, reduciendo el número de datos de la clase False, sin embargo, con esto perdemos información, o realmente estamos ignorando información para el entrenamiento, lo que puede provocar en que el entrenamiento sea pobre debido a la falta de datos.
>
>Con lo anterior podemos mejorar el desempeño de los modelos en clase True, balanceando el conjunto de datos. Además de esto, podemos mejorar las métricas mediante el ajuste de hiperparámetros de cada modelo.

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [X] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [X] Cuente cuantos por clase quedan.
- [X] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [X] Comente los resultados obtenidos.

In [15]:
# Agrupamos los tipos
mapping_dict = {
    'ST': 'ataque', 'CF': 'ataque', 
    'RW': 'central_ataque', 'CAM': 'central_ataque', 'LW': 'central_ataque', 
    'RM': 'central', 'CM': 'central', 'LM': 'central', 
    'RWB': 'central_defensa', 'CDM': 'central_defensa', 'LWB': 'central_defensa', 
    'RB': 'defensa', 'CB': 'defensa', 'LB': 'defensa', 
    'GK': 'arquero'
}

# Reemplazamos los tipos (los que no estén en el diccionario serán nulos)
df_players_2 = df_players.copy()
df_players_2['Club_Position'] = df_players_2['Club_Position'].map(mapping_dict)

# Eliminamos los valores nulos de la columna Club_Position pues es nuestra variable objetivo
df_players_2 = df_players_2.dropna(subset='Club_Position')
df_players_2

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Labels
0,Cristiano Ronaldo,Portugal,LS,central_ataque,185,80,Right,32,High / Low,4,...,95,85,92,93,90,81,76,85,88,True
1,Lionel Messi,Argentina,RW,central_ataque,170,72,Left,29,Medium / Medium,4,...,68,71,85,95,88,89,90,74,85,True
2,Neymar,Brazil,LW,central_ataque,174,68,Right,25,High / Medium,5,...,61,62,78,89,77,79,84,81,83,True
3,Luis Suárez,Uruguay,LS,ataque,182,85,Right,30,High / Medium,4,...,69,77,87,94,86,86,84,85,88,True
4,Manuel Neuer,Germany,GK,arquero,193,92,Right,31,Medium / Medium,4,...,78,25,25,13,16,14,11,47,11,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17298,Raymond Skjelde,Norway,NaN,defensa,185,75,Right,18,Medium / Medium,2,...,65,44,35,22,25,24,22,41,31,False
17301,Andrew O'Connor,Republic of Ireland,NaN,defensa,175,73,Left,21,Medium / Medium,3,...,55,43,26,28,29,30,30,31,20,False
17310,Tom Bolarinwa,England,NaN,central,180,80,Right,26,Medium / Medium,3,...,54,22,38,37,37,37,32,29,29,False
17312,Zak Mills,England,NaN,defensa,182,83,Right,24,Medium / Medium,4,...,61,43,22,24,28,30,32,39,31,False


In [16]:
# Contamos cuántos registros quedan por clase
df_players_2['Club_Position'].value_counts()

Club_Position
defensa            1180
central             907
arquero             632
central_ataque      581
ataque              430
central_defensa     209
Name: count, dtype: int64

In [17]:
# Eliminamos la columna Club_Position del preprocesamiento
onehot_columns = ['Preffered_Foot']

# Definimos el ColumnTransformer
preprocessing = ColumnTransformer(
    transformers = [
        ('standard', StandardScaler(), standard_columns),
        ('minmax', MinMaxScaler(), minmax_columns),
        ('onehot', OneHotEncoder(), onehot_columns),
        ('ordinal', OrdinalEncoder(categories=ordinal_categories), ordinal_columns),
    ],
    remainder = 'drop'
)

In [18]:
# Definimos las nuevas pipelines con los clasificadores
pipeline_1 = Pipeline([
    ('preprocessing', preprocessing), 
    ('classifier', LinearSVC(dual='auto', random_state=42))
])

pipeline_2 = Pipeline([
    ('preprocessing', preprocessing), 
    ('classifier', KNeighborsClassifier())
])

pipeline_3 = Pipeline([
    ('preprocessing', preprocessing), 
    ('classifier', SVC(random_state=42))
])

pipeline_4 = Pipeline([
    ('preprocessing', preprocessing), 
    ('classifier', DecisionTreeClassifier(random_state=42))
])

In [19]:
# Separamos el conjunto de datos (usamos stratify para mantener la distribución de labels)
features = df_players_2.drop(columns=["Club_Position"])
labels = df_players_2.loc[:, "Club_Position"]

X_train, X_test, y_train, y_test = train_test_split(
    features, labels, 
    test_size = 0.3, 
    stratify = labels,
    shuffle = True,
    random_state = 42,
)

In [20]:
# Entrenamos las pipelines
pipeline_1.fit(X_train, y_train)
pipeline_2.fit(X_train, y_train)
pipeline_3.fit(X_train, y_train)
pipeline_4.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('standard', StandardScaler(),
                                                  ['Height', 'Weight']),
                                                 ('minmax', MinMaxScaler(),
                                                  ['Age', 'Ball_Control',
                                                   'Dribbling', 'Marking',
                                                   'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Aggression', 'Reactions',
                                                   'Interceptions', 'Vision',
                                                   'Composure', 'Crossing',
                                                   'Short_Pass', 'Long_Pass',
                                                   'Acceleration', 'Speed',
                                                   'Stamina',...
                                                 ('onehot', OneHotEncoder(),
                                                  ['Preffered_Foot']),
                                                 ('ordinal',
                                                  OrdinalEncoder(categories=[['Low '
                                                                              '/ '
                                                                              'Low',
                                                                              'Low '
                                                                              '/ '
                                                                              'Medium',
                                                                              'Low '
                                                                              '/ '
                                                                              'High',
                                                                              'Medium '
                                                                              '/ '
                                                                              'Low',
                                                                              'Medium '
                                                                              '/ '
                                                                              'Medium',
                                                                              'Medium '
                                                                              '/ '
                                                                              'High',
                                                                              'High '
                                                                              '/ '
                                                                              'Low',
                                                                              'High '
                                                                              '/ '
                                                                              'Medium',
                                                                              'High '
                                                                              '/ '
                                                                              'High'],
                                                                             [1,
                                                                              2,
                                                                              3,
                                                                              4,
                                                                              5],
                                                                             [1,


In [21]:
# Predecimos 
predictions_1 = pipeline_1.predict(X_test)
predictions_2 = pipeline_2.predict(X_test)
predictions_3 = pipeline_3.predict(X_test)
predictions_4 = pipeline_4.predict(X_test)

In [22]:
for i in range(1, 5):
    predictions = globals()[f'predictions_{i}']
    print(f"Accuracy para pipeline_{i}: {accuracy_score(y_test, predictions)}")
    print(f"Cohen kappa para pipeline_{i}: {cohen_kappa_score(y_test, predictions)}\n")

Accuracy para pipeline_1: 0.7326565143824028
Cohen kappa para pipeline_1: 0.6578410509759233

Accuracy para pipeline_2: 0.643824027072758
Cohen kappa para pipeline_2: 0.5464110541010339

Accuracy para pipeline_3: 0.7098138747884941
Cohen kappa para pipeline_3: 0.6222748514910523

Accuracy para pipeline_4: 0.649746192893401
Cohen kappa para pipeline_4: 0.559637485725393



In [23]:
print(classification_report(y_test, predictions_1))

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       190
         ataque       0.72      0.89      0.80       129
        central       0.57      0.60      0.58       272
 central_ataque       0.45      0.25      0.32       174
central_defensa       0.56      0.16      0.25        63
        defensa       0.79      0.97      0.87       354

       accuracy                           0.73      1182
      macro avg       0.68      0.65      0.64      1182
   weighted avg       0.71      0.73      0.71      1182



In [24]:
print(classification_report(y_test, predictions_2))

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       190
         ataque       0.54      0.64      0.58       129
        central       0.46      0.51      0.48       272
 central_ataque       0.36      0.21      0.27       174
central_defensa       0.28      0.14      0.19        63
        defensa       0.76      0.86      0.81       354

       accuracy                           0.64      1182
      macro avg       0.57      0.56      0.55      1182
   weighted avg       0.62      0.64      0.63      1182



In [25]:
print(classification_report(y_test, predictions_3))

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       190
         ataque       0.72      0.78      0.75       129
        central       0.52      0.74      0.61       272
 central_ataque       0.00      0.00      0.00       174
central_defensa       0.00      0.00      0.00        63
        defensa       0.75      0.98      0.85       354

       accuracy                           0.71      1182
      macro avg       0.50      0.58      0.53      1182
   weighted avg       0.58      0.71      0.64      1182



C:\Users\Daniel Minaya Vargas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Daniel Minaya Vargas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Daniel Minaya Vargas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
print(classification_report(y_test, predictions_4))

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       190
         ataque       0.71      0.71      0.71       129
        central       0.48      0.47      0.48       272
 central_ataque       0.35      0.39      0.37       174
central_defensa       0.16      0.14      0.15        63
        defensa       0.81      0.80      0.80       354

       accuracy                           0.65      1182
      macro avg       0.59      0.58      0.58      1182
   weighted avg       0.65      0.65      0.65      1182



>Para este problema nos encontramos con algo similar al problema anterior. Los resultados son buenos en general para algunas clases, pero las clases con menos elementos son menos reconocidos por el modelo. En este caso tenemos que el LinearSVC es el modelo con mejor desempeño en todas las métricas. Para este modelo podemos ver que las clases central_defensa y central_ataque presentan un precision decente de alrededor 0.5, pero un recall muy pobre de alrededor 0.2, esto indica que cuando este modelo predice un jugador como central_defensa o central_ataque deberíamos confiar en esta predicción, sin embargo, no es capaz de detectar todos los jugadores de esta posición, es decir, muchos jugadores de esta posición serán predichos incorrectamente por el modelo.

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```salarios.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [X] Explique en que consiste la métrica $R^2$
- [X] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [X] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

>La métrica $R^2$ se utiliza en modelos de regresión y se conoce como *coeficiente de determinación*. Esta métrica proporciona una medida de cuánta variabilidad en la variable dependiente puede ser explicada por el modelo. Esta medida varía entre 0 y 1, donde valor 0 implica que el modelo no explica nada de la variabilidad de la variable dependiente alrededor de su media, mientras que valor 1 implica que el modelo explica toda la variabilidad de la variable dependiente alrededor de su media.

In [27]:
salary_players = pd.read_csv('salarios.csv')
salary_players = salary_players.rename(columns = {'Player': 'Name'})
salary_players

,Unnamed: 0,Name,Weekly Salary
0,1528,Lionel Messi,1538905.0
1,1006,Cristiano Ronaldo,1248536.0
2,1529,Antoine Griezmann,997032.0
3,2004,Neymar,797726.0
4,1530,Gareth Bale,607706.0
...,...,...,...
2529,1527,Simone Ghidotti,283.0
2530,2539,Valentin Gendrey,100.0
2531,1001,John Yeboah,70.0
2532,1000,Antonis Aidonis,70.0


In [28]:
new_df_players = pd.merge(df_players, salary_players, on='Name', how='inner')
new_df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Labels,Unnamed: 0,Weekly Salary
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,92,93,90,81,76,85,88,True,1006,1248536.0
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,85,95,88,89,90,74,85,True,1528,1538905.0
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,78,89,77,79,84,81,83,True,2004,797726.0
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,87,94,86,86,84,85,88,True,1531,508923.0
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,25,13,16,14,11,47,11,True,498,326233.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1861,Phillip Menzel,Germany,NaN,Res,191,83,Right,18,Medium / Medium,3,...,25,5,10,12,8,15,5,False,973,2034.0
1862,Manuel Akanji,Switzerland,NaN,Sub,187,85,Right,21,Medium / Medium,2,...,38,25,18,26,27,40,31,False,600,54176.0
1863,Moritz Nicolas,Germany,NaN,Res,195,87,Right,19,Medium / Medium,2,...,22,9,10,17,14,17,11,False,965,2262.0
1864,Giacomo Satalino,Italy,NaN,Sub,188,70,Right,17,Medium / Medium,1,...,19,6,5,13,11,18,9,False,1508,2827.0


In [29]:
# Agregamos Club_Position
onehot_columns = ['Club_Position', 'Preffered_Foot']

# Definimos el ColumnTransformer
preprocessing = ColumnTransformer(
    transformers = [
        ('standard', StandardScaler(), standard_columns),
        ('minmax', MinMaxScaler(), minmax_columns),
        ('onehot', OneHotEncoder(), onehot_columns),
        ('ordinal', OrdinalEncoder(categories=ordinal_categories), ordinal_columns),
    ],
    remainder = 'drop'
)

In [30]:
# Definimos las pipelines con los regresores
pipeline_1 = Pipeline([
    ('preprocessing', preprocessing), 
    ('classifier', LinearRegression())
])

pipeline_2 = Pipeline([
    ('preprocessing', preprocessing), 
    ('classifier', Lasso(alpha=1e3, random_state=42))
])

pipeline_3 = Pipeline([
    ('preprocessing', preprocessing), 
    ('classifier', Ridge(alpha=1e-1, random_state=42))
])

pipeline_4 = Pipeline([
    ('preprocessing', preprocessing), 
    ('classifier', SVR(kernel='linear', C=1e4))
])

In [31]:
# Separamos el conjunto de datos
features = new_df_players.drop(columns=["Weekly Salary"])
labels = new_df_players.loc[:, "Weekly Salary"]

X_train, X_test, y_train, y_test = train_test_split(
    features, labels, 
    test_size = 0.1, 
    shuffle = True,
    random_state = 42,
)

In [32]:
# Entrenamos las pipelines
pipeline_1.fit(X_train, y_train)
pipeline_2.fit(X_train, y_train)
pipeline_3.fit(X_train, y_train)
pipeline_4.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('standard', StandardScaler(),
                                                  ['Height', 'Weight']),
                                                 ('minmax', MinMaxScaler(),
                                                  ['Age', 'Ball_Control',
                                                   'Dribbling', 'Marking',
                                                   'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Aggression', 'Reactions',
                                                   'Interceptions', 'Vision',
                                                   'Composure', 'Crossing',
                                                   'Short_Pass', 'Long_Pass',
                                                   'Acceleration', 'Speed',
                                                   'Stamina',...
                                                 ('onehot', OneHotEncoder(),
                                                  ['Club_Position',
                                                   'Preffered_Foot']),
                                                 ('ordinal',
                                                  OrdinalEncoder(categories=[['Low '
                                                                              '/ '
                                                                              'Low',
                                                                              'Low '
                                                                              '/ '
                                                                              'Medium',
                                                                              'Low '
                                                                              '/ '
                                                                              'High',
                                                                              'Medium '
                                                                              '/ '
                                                                              'Low',
                                                                              'Medium '
                                                                              '/ '
                                                                              'Medium',
                                                                              'Medium '
                                                                              '/ '
                                                                              'High',
                                                                              'High '
                                                                              '/ '
                                                                              'Low',
                                                                              'High '
                                                                              '/ '
                                                                              'Medium',
                                                                              'High '
                                                                              '/ '
                                                                              'High'],
                                                                             [1,
                                                                              2,
                                                                              3,
                                                                              4,
                                                                              5],
             

In [33]:
# Predecimos 
predictions_1 = pipeline_1.predict(X_test)
predictions_2 = pipeline_2.predict(X_test)
predictions_3 = pipeline_3.predict(X_test)
predictions_4 = pipeline_4.predict(X_test)

for i in range(1, 5):
    predictions = globals()[f'predictions_{i}']
    print(f"R^2 score para pipeline_{i}: {r2_score(y_test, predictions)}\n")

R^2 score para pipeline_1: 0.13085119153072144

R^2 score para pipeline_2: 0.18016456015694549

R^2 score para pipeline_3: 0.13232947566951692

R^2 score para pipeline_4: 0.16686296414070634



>Para este modelo no se consiguió lograr un $R^2$ score de al menos 0.35, el modelo con el mayor $R^2$ score obtenido es el regresor Lasso con un 0.18 de score. Recordando que Lasso también realiza selección de atributos, el hecho de que sea el mejor modelo podría indicar que estamos utilizando muchos atributos para la regresión y quizás esa sea la razón por la cual los modelos no logran el desempeño esperado.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>